In [1]:
import numpy as np
import NUTS as hmc
import hmc as hmc2
import matplotlib.pyplot as plt
import corner
import time
import sklearn.datasets as skld

In [2]:
plt.rcParams['figure.figsize'] = [10, 10]

limits = [(-40,40),(-40,40)]
nparam = len(limits)

cov = skld.make_spd_matrix(nparam)
#cov = np.eye(nparam)
#cov = np.asarray([[1, 1.98],[1.98, 4]])
inv_cov = np.linalg.inv(cov)
print(cov)


[[2.19476956 0.58289318]
 [0.58289318 0.54992294]]


In [3]:
def mock_posterior_and_gradient(q):
    grad = -q @ inv_cov
    #j = theory_jacobian(p, *self.args).T
    logP = 0.5 * grad @ q.T
    #logP_jacobian = j @ dlogL_dCl
    #logP_jacobian = p * (-1)
    #mock_posterior_and_gradient.counter += 1
    return logP, grad

In [4]:
def run_hmc(n_it, hmc, cov, burnin):
    #rank = 5
    rank = nparam
    #np.random.seed(100 + rank)
    #C = np.eye(nparam)
    # mass matrix
    sampler = hmc.HMC(mock_posterior_and_gradient, cov, limits)
    # first sample starts at fid
    fid_params  = np.zeros(nparam)
    results = sampler.NUTS(fid_params, .8, n_it, burnin)

    return sampler

In [5]:
def run_hmc2(n_it, epsilon, hmc, spit, cov):
    #rank = 5
    rank = nparam
    #np.random.seed(100 + rank)
    #C = np.eye(nparam)
    # mass matrix
    sampler = hmc.HMC(mock_posterior_and_gradient, cov, epsilon, spit, limits)
    # first sample starts at fid
    fid_params  = np.zeros(nparam)
    results = sampler.sample(n_it, fid_params)

    return sampler

In [ ]:
nit = 10000
burnin = 5000

start = time.time()
chain = run_hmc(nit, hmc, cov, burnin)
print("Total time to run is %.2f s" %(time.time()-start))
print("")

start = time.time()
#chain2 = run_hmc(nit, hmc, cov, burnin)
chain2 = run_hmc2(nit, 0.02, hmc2, 50, cov)
print("Total time to run is %.2f s" %(time.time()-start))

0.5
Reasonable epsilon is 0.5
Step 750 of 15000
Step 1500 of 15000
Step 2250 of 15000
Step 3000 of 15000
Step 3750 of 15000
Step 4500 of 15000
final epsilon is 1.2997486116539145
Step 5250 of 15000
Step 6000 of 15000
Step 6750 of 15000
Step 7500 of 15000
Step 8250 of 15000
Step 9000 of 15000
Step 9750 of 15000
Step 10500 of 15000
Step 11250 of 15000
Step 12000 of 15000
Step 12750 of 15000
Step 13500 of 15000
Step 14250 of 15000
Total time to run is 10.13 s

Accept 0 alpha=1.00  p=0.00  ΔH=0.000


In [ ]:
#paths = np.array(chain.paths)
#anti_paths = np.array(chain.anti_paths)

trace = np.array(chain.trace)#[1::10])
calls = np.array(chain.ncall_list)#[1::10])

#trace = np.array(np.vstack({tuple(row) for row in chain.trace}))
#trace, idx = np.unique(np.array(chain.trace),True, axis=0)
#print(idx)
#calls = np.array(np.array(chain.ncall_list)[idx])

#trace = trace[idx.argsort()]
#calls = calls[idx.argsort()]

#print(trace)

print(trace.shape)
print(np.unique(trace[:,0]).shape)
print(calls.shape)

trace2 = np.array(chain2.trace)
calls2 = np.array(chain2.ncall_list)
print(trace2.shape)
print(np.unique(trace2[:,0]).shape)
print(calls2.shape)

In [ ]:
#plt.scatter(paths[:,0],paths[:,1],s=3)
#plt.scatter(anti_paths[:,0],anti_paths[:,1],s=3)
#plt.show()

plt.scatter(trace[:,0],trace[:,1],s=5)
#plt.scatter(trace2[:,0],trace2[:,1],s=5)
plt.show()

figure = corner.corner(trace)
figure2 = corner.corner(trace2)

In [ ]:
def plot_conv(trace,dim,botlim,toplim, hmc, cov, calls):
    STDs, means = [], []
    for i in range(trace.shape[0]):
        STD=np.std(trace[:i,dim])
        STDs.append(STD/np.sqrt(cov[dim,dim]))
        means.append(np.mean(trace[:i,dim]))
    plt.plot(calls, STDs, label="STD/sigma hmc %s" %hmc)
    plt.plot(calls, means, label="mean hmc %s" %hmc)
    plt.hlines([0,1],0,calls[-1],ls=":", color="k")
    plt.legend()
    plt.title("Mean & STD vs L-calls for dim %.0f" %dim)
    #plt.ylim(botlim,toplim)
    
    #print(np.std(trace[:,dim]))
    

In [ ]:
print(np.sqrt(cov))

plot_conv(trace,0,-1,1.5,1, cov, calls)
plot_conv(trace2,0,-1,1.5,2, cov, calls2)
plt.show()